<h1>Libraries</h1>

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
import re 
import pandas as pd
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import csv
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
n_words= set(stopwords.words('spanish'))
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud,STOPWORDS
porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<h1>X authentication</h1>

In [ ]:
def TwitterClient(): 
      # keys and tokens from the Twitter Dev Console 
      consumer_key = 'xxxxxxxx'
      consumer_secret ="xxxxxxxxxxx"
      access_token = 'xxxxxxxxxx'
      access_token_secret = 'xxxxxxxxxxx'

      # attempt authentication 
      try: 
          # create OAuthHandler object 
          auth = OAuthHandler(consumer_key, consumer_secret) 
          # set access token and secret 
          auth.set_access_token(access_token, access_token_secret) 
          # create tweepy API object to fetch tweets 
          api = tweepy.API(auth) 
      except: 
          print("Error: Authentication Failed") 

      return api #now we can make request to twitter using this api

<h1>Preprocessing text(cleaning)</h1>

In [ ]:
def clean(text):
  # removing @ tags and links from the text
  text= ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split()) 
  # converting all letters to lower case and relacing '-' with spaces.
  text= text.lower().replace('-', ' ')
  # removing stowards and numbers
  table= str.maketrans('', '', string.punctuation+string.digits)
  text= text.translate(table)
  # tokenizing words 
  tokens = word_tokenize(text)
  # stemming the words 
  stemmed = [porter.stem(word) for word in tokens]
  words = [w for w in stemmed if not w in n_words]

  text = ' '.join(words)
  return text

<h1>Analyzing sentiment</h1>

In [ ]:
analysis = TextBlob(tweet)
senti= analysis.sentiment.polarity

if senti<0 :
  emotion = "NEG"
elif senti>0:
  emotion= "POS"
else:
  emotion= "NEU"

<h1>Getting Tweets</h1>

In [ ]:
def get_tweets(query, count = 10): 
      tweets = [] 
      try: 
          # call twitter api to fetch tweets 
          fetched_tweets = api.search(q = query, count = count) 
          for tweet in fetched_tweets:
            # cleaning the tweets
            tweet= clean(tweet.text)
            # getting the sentiment from textblob
            analysis = TextBlob(tweet)
            senti= analysis.sentiment.polarity
            # labeling the sentiment
            if senti<0 :
              emotion = "NEG"
            elif senti>0:
              emotion= "POS"
            else:
              emotion= "NEU"
            # appending all data
            tweets.append((tweet, senti, emotion))
            
          return tweets
      except tweepy.TweepError as e:
          # print error (if any) 
          print("Error : " + str(e))

In [ ]:
# getting the api access
api = TwitterClient() 
# calling function to get tweets, count is the number of tweets.
tweets = get_tweets(query = "Farmer's Protest", count = 200)
df= pd.DataFrame(tweets, columns= ['tweets', 'senti', 'emotion'])
# droping retweets
df= df.drop_duplicates()


#save dataset in local
df.to_csv('tweets.csv', index= False)